<h1>Estrategias y planteamientos:</h1>

Al ver el problema y los datos que se daba, pensé en solucionarlo de diversas maneras, siendo estas principalmente entrenar una IA a través de etiquetas ya puestas, definir un número de grupo y que la IA sortée, o en un modelo de NLP establecer las entidades para que los mails se categorizasen en base a la interpretación de este. Al hablar con rrhh por la dirección ala que debería ir, me dijeron que era mejor si mostraba más mis conocimientos y habilidades, e imagine que por la descripción del puesto sería el más adecuado. Me he decantado por usar sklearn pues es una biblioteca de aprendizaje automático de Python con varios algoritmos y es sencillo y eficiente a la hora de usarlo.

He decidido estructurar en dos el código, tenemos por un lado el modelo.py, que se lanza de forma manual para que se cree y entrene el categorizador de forma previa y así evitar el tiempo de espera por respuesta que recibamos, y la API, que, tras recibir una llamada, comprueba si es de alguien que sea o no fraudulento y en caso de que no lo sea categoriza y devuelve esta(s) como respuesta. 

La división es en estas distintas categorias: "Facturas", "Problemas tecnicos", "Energia", "Gas" y "Contrato" ya que cada cual contiene suficientes muestras. Consideré añadir "Lecturas" dentro pero la falta de correos de ese tipo me llevó a no hacerlo. En caso de haber usado PNL lo hubiese añadido como categoria con sus entidades, pero al ser ML simplemente añadiria datos no deseables.


Ahora, pasando al análisis de datos, nos conectaremos a la BBDD y buscaremos los distintos correos por categorias para buscar correlación.

In [134]:
import mysql.connector

conexion = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='root',
    database='atc'
)

cursor = conexion.cursor()

In [135]:
consulta =  '''
            SELECT
                clientes.nombre,
                emails.fecha_envio,
                GROUP_CONCAT(categorias.nombre) AS categorias,
                emails.email
            FROM
                emails
            INNER JOIN clientes ON emails.client_id = clientes.id
            LEFT JOIN emails_categorias ON emails.id = emails_categorias.email_id
            LEFT JOIN categorias ON emails_categorias.categoria_id = categorias.id
            GROUP BY
                clientes.nombre, emails.fecha_envio, emails.email
            ORDER BY
                clientes.nombre, emails.fecha_envio
            '''

cursor.execute(consulta)
resultado = cursor.fetchall()

print(f"{'Fecha Envío':<20}{'Categorías':<20}{'Email':<50}")

# Formatear y mostrar los datos en el notebook
current_nombre = None

for row in resultado:
    nombre = str(row[0])
    fecha_envio = str(row[1])
    categorias = str(row[2])
    email = row[3]

    if nombre != current_nombre:
        print(f"\nCliente: {nombre}")
        current_nombre = nombre

    print(f"{fecha_envio:<20}{categorias:<20}{email:<50}\n\n")


Fecha Envío         Categorías          Email                                             

Cliente: Arnau
2022-05-08 08:54:34 Facturas,Gas,EnergiaHola, buenos dÃ­as. Mira que me habÃ©is cobrado las facturas de la luz y el gas, pero no me habÃ©is mandado la factura. Â¿Vale? Gracias.


2022-08-08 19:22:27 Facturas            Buenos dÃ­as. Quisiera que me enviarÃ¡n tambiÃ©n las siguientes facturas:

- recibida 18/07/2023, importe XXâ‚¬
- recibida 18/07/2023, importe YYâ‚¬
- recibida 18/08/2023, importe ZZâ‚¬


2022-08-13 06:56:49 Facturas            Buenos dÃ­as, Necesitamos que nos envÃ­en la factura del mes de septiembre del siguiente cliente: XXXXXX



Cliente: Berta
2022-05-23 19:11:41 Facturas,Contrato,Energia

Buenos dias , nos ponemos en contacto con ustedes para solicitar documentaciÃ³n necesaria para la central :Â 

TITULAR : XXXX PRUEBA
NIF B123456789

REF CONTRATO :XXXXXXXX


Contrato de electricidad vigente, Facturas de energÃ­a de los Ãºltimos 12 meses, Curvas de consumo del

Primero de todo, los correos que hemos usado para analizar y entrenar nuestro modelo, podemos ver que la media ronda por 3 por cabeza, con unos outliers de 2 y 4. Estos correos se les ha asignado las categorias previamente mencionadas, y con ello se ha entrenado el modelo, a continuación la densidad de cada categoria por mail:

In [136]:
consulta = "SELECT COUNT(*) FROM emails"
cursor.execute(consulta)
num_emails_total = cursor.fetchone()[0]

consulta =  '''
            SELECT
                categorias.nombre AS categoria,
                COUNT(emails_categorias.email_id) AS num_emails
            FROM
                categorias
            LEFT JOIN emails_categorias ON categorias.id = emails_categorias.categoria_id
            GROUP BY
                categorias.nombre
            ORDER BY
                categorias.nombre
            '''

cursor.execute(consulta)
resultado = cursor.fetchall()

print(f"{'Categoría':<30}{'Cantidad de Mails':<20}{'Porcentaje de mails':<10}")

for row in resultado:
    category = row[0]
    count = row[1]
    perc = count/num_emails_total
    print(f"{category:<30}{count:<20}{perc:<5}")


Categoría                     Cantidad de Mails   Porcentaje de mails
Contrato                      8                   0.34782608695652173
Energia                       9                   0.391304347826087
Facturas                      16                  0.6956521739130435
Gas                           8                   0.34782608695652173
Problemas tecnicos            10                  0.43478260869565216


Podemos ver que la mayoria de los mails se tratan de facturas, seguidos por aquellos con problemas técnicos (sean de web, de no poder abrir una factura o recibirla, etc), mientras que tanto los temas de contrato, energia y gas tienen suficiente peso como para que se considere a la hora de asignar categoria.

In [137]:
consulta = '''
            SELECT
                categorias.nombre AS categoria,
                emails.fecha_envio
            FROM
                categorias
            LEFT JOIN emails_categorias ON categorias.id = emails_categorias.categoria_id
            LEFT JOIN emails ON emails_categorias.email_id = emails.id
            ORDER BY
                categorias.nombre, emails.fecha_envio
            '''

cursor.execute(consulta)
resultado = cursor.fetchall()

# Formatear y mostrar los datos en el notebook
print(f"{'Categoria':<20}{'Fecha Envío':<50}")
i = 0
for row in resultado:
    categoria = str(row[0])
    fecha_envio = str(row[1])
    i+=1
    print(f"{categoria:<20}{fecha_envio:<50}")
print(i)

Categoria           Fecha Envío                                       
Contrato            2022-01-05 06:49:43                               
Contrato            2022-01-25 19:57:15                               
Contrato            2022-02-18 19:29:31                               
Contrato            2022-04-18 07:11:29                               
Contrato            2022-05-23 19:11:41                               
Contrato            2022-08-08 20:42:46                               
Contrato            2022-08-12 08:52:39                               
Contrato            2022-11-09 20:40:38                               
Energia             2022-01-05 06:49:43                               
Energia             2022-03-15 08:38:44                               
Energia             2022-04-07 19:25:24                               
Energia             2022-05-08 08:54:34                               
Energia             2022-05-12 20:01:26                               
Energi

Los datos están muy esparcidos, pero podemos ver que tanto a principio de año los mails de contrato tienen mayor cuantia, al igual que final de año. Los temas de gas abundan entre marzo, abril y mayo, lo cual tiene sentido pues las facturas de gas de invierno tienden a llegar hacia esas fechas. Y de la misma forma, los temas de facturas están también por esas fechas. Tanto los temas de energia como de problemas técnicos están esparcidos de forma equitativa por el año.

In [140]:
consulta =  '''
            SELECT
                categorias.nombre AS categoria,
                COUNT(DISTINCT emails_categorias.email_id) AS num_emails,
                GROUP_CONCAT(DISTINCT emails.fecha_envio ORDER BY emails.fecha_envio) AS emails_fecha
            FROM
                impagos
            INNER JOIN emails ON impagos.client_id = emails.client_id
            INNER JOIN emails_categorias ON emails_categorias.email_id = emails.id
            INNER JOIN categorias ON categorias.id = emails_categorias.categoria_id
            GROUP BY
                categorias.nombre
            ORDER BY
                categorias.nombre;
            '''

cursor.execute(consulta)
resultado = cursor.fetchall()

print(f"{'Categoría':<30}{'Cantidad de Mails':<20}{'Porcentaje de mails':<20}\t{'Fecha':<20}\n\n")

for row in resultado:
    category = row[0]
    count = row[1]
    perc = count/6
    fecha = row[2]
    print(f"{category:<30}{count:<20}{perc:<5}\t{fecha:<20}\n")



Categoría                     Cantidad de Mails   Porcentaje de mails 	Fecha               


Contrato                      1                   0.16666666666666666	2022-08-12 08:52:39 

Energia                       3                   0.5  	2022-05-08 08:54:34,2022-07-13 18:21:44,2022-08-12 08:52:39

Facturas                      5                   0.8333333333333334	2022-05-08 08:54:34,2022-06-12 06:23:15,2022-07-13 18:21:44,2022-08-08 19:22:27,2022-08-13 06:56:49

Gas                           2                   0.3333333333333333	2022-05-08 08:54:34,2022-08-12 08:52:39

Problemas tecnicos            1                   0.16666666666666666	2022-06-12 06:23:15 



Contemplando lo previamente indicado, vemos que el caso de los impagos no sigue un patrón específico, pero sí que es de notar que la energia es un outlier, pues antes era baja la frecuencia y ha aumentado de un 0.3913 a un 0.5, y la cantidad de las facturas es mayor incluso, pasando de 0.6956 a 0.8333. Respecto a las fechas, vemos que están más cerca del verano que otra cosa.

In [ ]:
cursor.close()
conexion.close()

Finalmente, cerramos la conexión con la BBDD